### multi class label 시도

In [1]:
import pandas as pd

# 식품 섭취 데이터는 사용하지 않는 것으로 결정(2018-11-15)
df_ALL = pd.read_sas("./HN16_ALL.sas7bdat", format = 'sas7bdat', encoding='iso-8859-1')
# df_ffq = pd.read_sas("./hn16_ffq.sas7bdat", format = 'sas7bdat', encoding='iso-8859-1')

# 19세 이상으로만 제한
df_data = df_ALL.loc[(18 < df_ALL.age), :]
# df_data.shape

# 혈압치료중인 대상 제외
df_data = df_data.loc[(1 < df_data.DI1_pt) | (df_data.DI1_pt < 1), :]
# df_data.shape

# 모든 값이 NaN값이 컬럼 삭제
df_data = df_data.dropna(axis=1, how='all')
# df_data.shape

# 고혈압 유병여부 NaN값 대상 삭제
df_data = df_data.dropna(subset=['HE_HP'])
# df_data.shape

# index 재 생성을 위해 ID로 인덱스 생성 후 인덱스 삭제
df_data =  df_data.set_index('ID')
df_data = df_data.reset_index()

# 고혈압 유병여부
# 1. 정상
# 2. 고혈압전단계
# 3. 고혈압
# classification을 위해3. 고혈압을 2.로 변경하여 이산형으로 수정

#df_data.loc[df_data.HE_HP == 3.0, 'HE_HP'] = 2.0
print("HE_HP :\n")
print(pd.value_counts(df_data.HE_HP.values, sort=False).sort_index()) # 고혈압 유병 여부

HE_HP :

1.0    2640
2.0    1468
3.0     600
dtype: int64


In [23]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn import tree
import graphviz
from tqdm import tqdm_notebook

#x_name = ['age', 'sex']
# x_name = ['HE_ht']
# y_name = 'HE_HP'

def saveDTR(input_df, x_name, y_name) :

    arrX1 = df_data[[x_name]]
    arrY = df_data[y_name]

    arrX1 = arrX1.fillna(-1)
    #print(arrX1.isnull().sum())

    model = DecisionTreeClassifier(
        criterion = 'gini',
        max_depth=6
    )
    #print(model)

    model.fit(arrX1,arrY)

    dot_data = tree.export_graphviz(model, out_file=None, 
                                    feature_names=[x_name], 
                                    class_names=['정상', '고혈압전단계', '고혈압']
                                   )

    graph = graphviz.Source(dot_data) 
    graph.render("./pdf_20181117/%s+%s" % (x_name,y_name))
    #graph.render("%s+%s" % (x_name,y_name))

#     modelPrediction = model.predict(arrX1)
#     accuracyRate = accuracy_score(
#         y_true=arrY
#     ,   y_pred=modelPrediction
#     )
#     accuracyRate_list.append([x_name, y_name, accuracyRate])

    #print('정확도 = ', accuracyRate)

# # HE_HP(고혈압 유병여부) 제외
# skip_list = [
#     df_data.HE_HP.name
#     #, df_data.ID.name
    
# ]

# accuracyRate 담을 변수 준비
# accuracyRate_list = []

saveDTR(df_data, "N_K", df_data.HE_HP.name)

# for x_name in tqdm_notebook(df_data.columns):
#     if x_name in skip_list : continue
#     #print(df_data[x_name].dtype.name)
#     if df_data[x_name].dtype == 'object' : continue
#     saveDTR(df_data, x_name, df_data.HE_HP.name)
    
# # accuracyRate_list > dataframe으로 변환해서 csv파일로 저장   
# df_accuracyRate = pd.DataFrame(accuracyRate_list)
# df_accuracyRate.to_csv("accuracyRate_list_multi_class.csv")